# MAP 435 - TP à trous à rendre pour le dimanche 12 mai 23h59

Le but de cet exercice est de compléter les fonctions manquantes en remplaçant les parties `# YOUR CODE HERE` par votre code.

**Attention** la correction des notebooks se faisant de manière automatique :
- il faut que vous supprimiez la ligne `raise NotImplementedError()` ;
- les seules cellules que vous devez modifier sont celles comportant la mention `# YOUR CODE HERE` ;
- **vous ne devez pas modifier le nom du fichier faute de quoi votre devoir ne sera pas corrigé**.

Merci d'inscrire votre nom dans la cellule ci-dessous.

In [17]:
NOM = "MOK Yong"

# Amélioration des méthodes de gradient

Le but de cet exercice est d'améliorer la convergence de l'algorithme du gradient à pas optimal classique décrit dans votre polycopié par une méthode que l'on explicitera plus loin.


## Chargement des librairies utilisées

In [18]:
import numpy as np
import numpy.testing as npt
import matplotlib.pyplot as plt
%matplotlib inline

import scipy as sp
from scipy import sparse
from scipy.sparse import linalg

## Algorithmes 

### Algorithme du gradient pas optimal

Vous trouverez ci-dessous l'algorithme du gradient à pas optimal. Cet algorithme est complet et **vous n'avez pas à le modifier**. Il est simplement présent afin de comparer le résultat que vous obtiendrez avec l'algorithme du gradient à pas optimal préconditionné.

On construit la suite $x_k$ définie par :

\begin{equation}
\left\{
\begin{aligned}
&x_0 \in \mathbb R^N \text{ quelconque} \\
&x_{k+1} = x_k - \mu_k \nabla J(x_k),
\end{aligned}
\right.
\end{equation}

où le pas de descente $\mu_k$ est choisi tel que :

$$ J(x_{k+1})=\min_{\mu \in \mathbb R} J(x_k-\mu\nabla J(x_k)).$$

Lorsqu'on travaille avec une fonctionnelle quadratique  $J: \mathbb R^N \rightarrow \mathbb R$ telle que  
$$ J(x) = \frac12 \langle Ax,x\rangle - \langle b,x\rangle,$$
avec $A$ une matrice symétrique définie positive, on sait que le pas de descente optimal est donné par 
$$\mu_k=\frac{\|r_k\|^2}{\left\langle r_k,Ar_k\right\rangle} \text{ avec } r_k= Ax_k-b=\nabla J(x_k).$$

Par souci de simplicité on donne seulement ici l'algorithme du gradient à pas optimal dans le cas d'une fonctionnelle quadratique. Ainsi les arguments sont : la fonctionnelle à minimiser `f`, la matrice $A=$ `mat_A`, le vecteur $b=$ `vec_b` et un point initial `x_init`. 
On y ajoute les arguments optionnels suivants : la tolérance demandée `tol` et le nombre maximal d'itérations `maxiter`.

In [19]:
def gradientPasOpt(f,mat_A,vec_b,x_init,tol=1e-06,maxiter=200):
    
    # initialisation
    x=x_init.copy()
    xtab=[]
    ftab=[]
    
    xtab.append(x) # on ajoute x à la liste xtab
    ftab.append(f(x))
    
    it=0 # compteur d'itération
    
    while((it==0) or (it<maxiter and np.linalg.norm(mat_A@xtab[-1]-vec_b)>tol)): 
        
        r=mat_A@x-vec_b
        pas = np.dot(r,r)/np.dot(r,mat_A@r)
 
        x=x-pas*r
        
        xtab.append(x)
        ftab.append(f(x))
        
        it=it+1
    
    # booléen pour indiquer la convergence
    if(it==maxiter):
        conv = False
    else:
        conv = True
    
    return xtab, ftab, conv

### Gradient à pas optimal préconditionné

On souhaite maintenant améliorer la vitesse de convergence de cet algorithme. Pour cela on va utiliser un préconditionnement. 
Dans ce cas là, si $P$ est la matrice de préconditionnement, l'algorithme de gradient à pas optimal devient :
$$x_{k+1} = x_k-\mu_k P^{-1} \nabla J(x_k).$$


On vient de voir que dans le cas d'une fonctionnelle quadratique $ J(x) = \frac12 \langle Ax,x\rangle - \langle b,x\rangle$ avec $A$ symétrique définie positive, le pas de descente optimal dans l'algorithme du gradient à pas optimal (sans préconditionnement) est donné par :
$$\mu_k=\frac{\|r_k\|^2}{\left\langle r_k,Ar_k\right\rangle} \text{ avec } r_k= Ax_k-b.$$

On rappelle également que ce pas optimal a été obtenu en minimisant la fonction $\rho \mapsto J(x_k-\rho\nabla J(x_k))$.

**Q1)** Après avoir calculé (à la main) la nouvelle valeur du pas optimal dans le cas du gradient à pas optimal préconditionné (pour le cas d'une fonctionnelle quadratique), complétez  le code ci-dessous qui permet de calculer le pas optimal  en fonction du résidu $r_k=\nabla J(x_k) = Ax_k-b=$ `r`, de la matrice du problème $A=$ `mat_A` et de la matrice $C=P^{-1}=$ `mat_C`.


In [20]:
def calcul_pas_precond(r, mat_A, mat_C):
    # Preconditioned residual
    Cr = mat_C@r
    
    # Applying A to the preconditioned residual
    A_Cr = mat_A@Cr
    
    # Calculating the optimal step size mu_k
    mu_k = np.dot(r, Cr) / np.dot(Cr, A_Cr)
    
    return mu_k
    

**Q2)** Complétez  le code ci-dessous qui permet de calculer l'itérée $x_{k+1}$ en fonction de l'itérée $x_k$ pour l'algorithme du gradient à pas optimal préconditionné.

Les arguments de cette fonction sont : la valeur de l'itérée $x_k$ notée `x`, le gradient de la fonctionnelle à minimiser qui est égal au résidu `r`, la matrice de préconditionnement `mat_C` ainsi que le pas `pas`.

In [21]:
def calcul_xplus(x, r, mat_C, pas):
    x = x.copy()
    # Calculate C times r
    Cr = mat_C@r
    
    # Calculate the update step
    update = pas * Cr
    
    # Update x to get x_k+1
    x_k_plus = x - update
    
    return x_k_plus

On donne maintenant l'algorithme du gradient à pas optimal préconditionné dans lequel l'itérée $x_{k+1}$ et le pas optimal sont obtenus en utilisant les fonctions`calcul_xplus` et `calcul_pas_precond` que vous venez de construire.

Les arguments de cette fonction sont : la fonctionnelle à minimiser `f`, la matrice $A=$ `mat_A`, le vecteur $b=$ `vec_b`, la matrice `mat_C` et un point initial `x_init`. On y ajoute les arguments optionnels suivants : la tolérance demandée `tol` et le nombre maximal d'itérations `maxiter`.

In [22]:
def gradientPasOpt_precond(f,mat_A,vec_b,mat_C,x_init,tol=1e-06,maxiter=200):
    
    # initialisation
    x=x_init.copy()
    xtab=[]
    ftab=[]
    
    xtab.append(x) # on ajoute x à la liste xtab
    ftab.append(f(x))
    
    it=0 # compteur d'itération
    
    while((it==0) or (it<maxiter and np.linalg.norm(mat_A@xtab[-1]-vec_b)>tol)):
           
        r = mat_A@x-vec_b
        pas = calcul_pas_precond(r, mat_A, mat_C)
            
        x = calcul_xplus(x, r, mat_C, pas)
        
        xtab.append(x)
        ftab.append(f(x))
        
        it = it+1
    
    # booléen pour indiquer la convergence
    if(it==maxiter):
        conv = False
    else:
        conv = True
    
    return xtab, ftab, conv

## Définition du problème

On va chercher à minimiser la fonctionnelle quadratique suivante $J: \mathbb R^N \rightarrow \mathbb R$ telle que  
$$ J(x) = \frac12 \langle Ax,x\rangle - \langle b,x\rangle,$$
avec $A$ la matrice symmétrique telle que pour $0 \le i \le N-1$,
$$A_{i,i}=3i^2+8, \quad A_{i,i+1}=A_{i,i-1}=i+1 \ \text{ et } \ A_{i,i+2}=A_{i,i-2}=-1$$
et $b=\left(i(N-1-i)\right)_{0 \le i \le N-1}$.

On rappelle que trouver le minimum de la fonctionnelle $J$ revient alors à résoudre le système linéaire $Ax=b$.


**Q3)** Complétez le programme ci-dessous qui pour `N` donné renvoit le vecteur $b$ proposé ci-dessus de taille `N`.

In [23]:
def fct_vecb(N):
    # YOUR CODE HERE
    b = [(i*(N-1-i)) for i in range(N)]
    return np.array(b)

In [24]:
# on affiche le vecteur que vous avez construit.
print(fct_vecb(21))


[  0  19  36  51  64  75  84  91  96  99 100  99  96  91  84  75  64  51
  36  19   0]


**Q4)** Complétez le programme ci-dessous qui pour `N` donné renvoit la matrice carrée de taille `N` **creuse** $A$ décrite ci-dessus. 
**Faites attention à bien construire une matrice creuse.**

In [25]:
def fct_matA(N):
    # YOUR CODE HERE
    A_diag = [(3 * i**2 + 8) for i in range(N)]
    off_diag1 = [(i + 1) for i in range(0, N)]
    off_diag2 = [-1 for i in range(2, N)]
    A = sparse.diags(
        [off_diag2, off_diag1, A_diag, off_diag1, off_diag2],
        offsets=[-2, -1, 0, 1, 2],
        shape=(N, N),
        format='csr'  # Compressed Sparse Row format
    )
    return A
    

In [26]:
# on affiche la matrice que vous avez construite
print(fct_matA(10).todense())


[[  8.   1.  -1.   0.   0.   0.   0.   0.   0.   0.]
 [  1.  11.   2.  -1.   0.   0.   0.   0.   0.   0.]
 [ -1.   2.  20.   3.  -1.   0.   0.   0.   0.   0.]
 [  0.  -1.   3.  35.   4.  -1.   0.   0.   0.   0.]
 [  0.   0.  -1.   4.  56.   5.  -1.   0.   0.   0.]
 [  0.   0.   0.  -1.   5.  83.   6.  -1.   0.   0.]
 [  0.   0.   0.   0.  -1.   6. 116.   7.  -1.   0.]
 [  0.   0.   0.   0.   0.  -1.   7. 155.   8.  -1.]
 [  0.   0.   0.   0.   0.   0.  -1.   8. 200.   9.]
 [  0.   0.   0.   0.   0.   0.   0.  -1.   9. 251.]]


On choisit ici d'utiliser un préconditionnement SOR (Successive Over-Relaxation), c'est à dire que la matrice de précondionnement est donnée par :
$$P=\omega\left(D+\omega L\right), \quad \omega >1,$$
où $D,L$ sont des matrices carrées de taille $N$ telle que :
- $D$ représente la diagonale de la matrice $A$ ;
- $L$ est la partie triangulaire inférieure (sans la diagonale) de la matrice $A$.

**Q5)** Complétez la fonction ci-dessous qui pour $\omega=$ `omega` et `N` donnés renvoit la matrice **creuse** $P$ décrite ci-dessus. 
**Faites attention à bien construire une matrice creuse.**
DEF D et L

In [27]:
def fct_matP(N,omega):
    # YOUR CODE HERE
    A = fct_matA(N)
    # Extract D as a diagonal matrix
    D = sparse.diags(A.diagonal(), 0, shape=(N, N), format='csr')
    
    
    # Extract L (strictly lower triangular part of A)
    L = sparse.tril(A, k=-1)
    # Construct the preconditioning matrix P
    P = omega * (D + omega * L)
    return P

In [28]:
# on affiche la matrice que vous avez construite
omega=1.2
print(fct_matP(10,omega).todense())


[[  9.6    0.     0.     0.     0.     0.     0.     0.     0.     0.  ]
 [  1.44  13.2    0.     0.     0.     0.     0.     0.     0.     0.  ]
 [ -1.44   2.88  24.     0.     0.     0.     0.     0.     0.     0.  ]
 [  0.    -1.44   4.32  42.     0.     0.     0.     0.     0.     0.  ]
 [  0.     0.    -1.44   5.76  67.2    0.     0.     0.     0.     0.  ]
 [  0.     0.     0.    -1.44   7.2   99.6    0.     0.     0.     0.  ]
 [  0.     0.     0.     0.    -1.44   8.64 139.2    0.     0.     0.  ]
 [  0.     0.     0.     0.     0.    -1.44  10.08 186.     0.     0.  ]
 [  0.     0.     0.     0.     0.     0.    -1.44  11.52 240.     0.  ]
 [  0.     0.     0.     0.     0.     0.     0.    -1.44  12.96 301.2 ]]


## Observation des résultats

On compare alors l'algorithme du gradient optimal préconditionné au gradient optimal classique.

Les résultats obtenus ci-dessous doivent vous permettre de valider ou d'invalider vos codes.

In [29]:
N=300

# Vecteur b
vec_b=fct_vecb(N)

# Matrice A
mat_A=fct_matA(N)


# Calcul de la solution
sol=linalg.spsolve(mat_A.tocsc(), vec_b, use_umfpack=True)

# Fonctionnelle
def fct_J(x):
    return 0.5*np.dot(x,mat_A@x)-np.dot(vec_b,x)

def fct_gradJ(x):
    return mat_A@x-b


On peut maintenant construire la matrice utilisée dans l'algorithme de gradient optimal préconditionné.

In [30]:
mat_P = fct_matP(N,1.2)
mat_C=sp.sparse.csc_matrix(mat_P)
mat_C=sp.sparse.linalg.inv(mat_C)

In [31]:
x_init=np.ones(N)

print('=================================')
print('Algorithme du gradient à pas optimal')
x_PasOpt_sansprecond, f_PasOpt, conv_PasOpt = gradientPasOpt(fct_J,mat_A,vec_b,x_init,1e-6,50000)
print('Erreur = ',np.linalg.norm(sol-x_PasOpt_sansprecond[-1]))
print('Convergence =',conv_PasOpt)
print('Nombre d itérations = ',np.shape(x_PasOpt_sansprecond)[0]-1)


print('=================================')
print('Algorithme du gradient à pas optimal préconditionné')
x_PasOpt_precond, f_PasOpt, conv_PasOpt = gradientPasOpt_precond(fct_J,mat_A,vec_b,mat_C,x_init,1e-6,5000)
print('Erreur = ',np.linalg.norm(sol-x_PasOpt_precond[-1]))
print('Convergence =',conv_PasOpt)
print('Nombre d itérations = ',np.shape(x_PasOpt_precond)[0]-1)


Algorithme du gradient à pas optimal
Erreur =  0.48892525798778164
Convergence = False
Nombre d itérations =  50000
Algorithme du gradient à pas optimal préconditionné
Erreur =  3.8346799337732045e-08
Convergence = True
Nombre d itérations =  8
